<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
ДАМИР, рад приветствовать ВАС! <br />
<br />    

<br />
Меня зовут Николай. <br />И на этом этапе Вашего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />
<font color='green'>Моей целью является не "уличить" в не знании, а просто высказать сверху Ваших знаний - свой опыт, что бы Вы могли использовать его для Вашего дальнейшего успеха)
<br>
</font>
<br>

</font>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd



In [7]:
data = pd.read_csv('/datasets/users_behavior.csv')


In [9]:
data.shape

(3214, 5)

In [10]:
data.head()

calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

Датасет состоит из 3 214 объектов и 5 признаков. Целевой признак для нашей задачи – is_ultra, т.е. модель, которую мы попытаемся построить, будет предсказывать значение 1 если клиенту нужно предложить тариф "Ультра" или 0 если тариф "Смарт". В этом и заключается задача классификации, т.к. наш целевой признак является категориальным.

## 2. Разбейте данные на выборки

Спрятанной тестовой выборки в задании нет, поэтому может разбить исходный набор данных на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов сделаем равными.



In [12]:
# Зафиксируем псевдослучайность для всех используемых в проекте алгоритмов
rnd_state = 202006

In [25]:
# 60% отводим под обучающую выборку
df_train, df_valid = train_test_split(data, test_size=0.4, random_state=rnd_state)
# половину из оставшихся 40% отдаем на валидационную, другую половину – на тестовую
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=rnd_state)


В итоге получили следующее соотношение: 60% | 20% | 20%



Чуть позднее, после проверки модели на валидационной выборке выполним оценку ещё и на тестовом наборе. Это позволит правильно оценить готовую модель.



<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
ОК

</b></font>

## 3. Исследуйте модели

Для решения задачи классификации рассмотрим следующие изученные модели:

- дерево решений / decision tree
- случайный лес / random forest
- логистическую регрессию / logistic regression

In [26]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [27]:
def accuracy_score_valid(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

# Дерево решений

In [28]:
# Посмотрим как глубина решающего дерева влияет на оценку правильности
for max_depth in range(1, 21, 2):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_state)
    model.fit(features_train, target_train)
    print(f"max_depth = {max_depth}:\t{accuracy_score_valid(model)}")

max_depth = 1:	0.76049766718507
max_depth = 3:	0.807153965785381
max_depth = 5:	0.8087091757387247
max_depth = 7:	0.8009331259720062
max_depth = 9:	0.8009331259720062
max_depth = 11:	0.7931570762052877
max_depth = 13:	0.7791601866251944
max_depth = 15:	0.7636080870917574
max_depth = 17:	0.7558320373250389
max_depth = 19:	0.744945567651633


 Судя по валидационной выборке, дерево решений имеет самую высокую оценку правильности 0.8087091757387247, когда задан гиперпараметр глубины равный 5.

# Случайный лес 

In [29]:

# В качестве гиперпараметра глубины дерева возьмем значение, найденное для предыдущей модели.
# А количество деревьев для нашего случайного леса будет искать в диапазоне от 10 до 100 с шагом 10.
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=5, random_state=rnd_state)
    model.fit(features_train, target_train)
    print(f"n_estimators = {estim}:\t{accuracy_score_valid(model)}")

n_estimators = 10:	0.8149300155520995
n_estimators = 20:	0.8164852255054432
n_estimators = 30:	0.8180404354587869
n_estimators = 40:	0.8164852255054432
n_estimators = 50:	0.8149300155520995
n_estimators = 60:	0.8164852255054432
n_estimators = 70:	0.8164852255054432
n_estimators = 80:	0.8149300155520995
n_estimators = 90:	0.8164852255054432
n_estimators = 100:	0.8164852255054432


📝 Модель случайного леса предсказывает тариф точнее, но, как мы видим, не на много – 0.8180404354587869, даже при количестве деревьев леса равным 30.



# Логистическая регрессия 


In [30]:
def logistic_regression(solver='liblinear'):
    model = LogisticRegression(solver=solver, random_state=rnd_state)
    model.fit(features_train, target_train)
    return accuracy_score_valid(model)

In [31]:
logistic_regression()


0.7169517884914464

In [32]:
solvers = ['newton-cg', 'lbfgs']


In [33]:
for solver in solvers:
    print(f'{solver}: {logistic_regression(solver=solver)}')

newton-cg: 0.7589424572317263
lbfgs: 0.7169517884914464


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Даже используя другой алгоритм для решения задачи оптимизации модель логистической регрессии менее точна, чем дерево решений и случайный лес.

## 4. Проверьте модель на тестовой выборке

В результате наших экспериментов на валидационной выборке мы выяснили, что самую высокую оценку правильности дает лес решений с глубиной равной 7 и количеством деревьев равным 70.



In [34]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

model = RandomForestClassifier(n_estimators=30, max_depth=5, random_state=rnd_state)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.7869362363919129

На тестовой выборке наша модель случайного леса показывает приближенное к тренировочной выборке значение accuracy. Таким образом, мы не видим здесь явной проблемы с переобучением, и можем считать, что нам удалось построить модель для задачи бинарной классификации, которая выберет подходящий тариф.



<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
понятно, ок
<br>
</font>

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
<br/> Дамир,  <br>
работа такая, что не разгнаться взглядом)))))<br>
Всё логично, адекватно, вдумчиво и с выводами - это важно.<br>
организовали перебор параметров моделей.....
Мне и добавить то нечего)<br>

Отмечу только один момент, так для практики (этого не было в задании)<br>
На практике, совместно с AUC (это ведь лишь обобщённая метрика), очень полезно использовать метрики "полнота" "точность" TruePositive....
Это поможет забраковать модели с высоким AUC но низкой бизнес-полезностью. Например, модель очень хорошо предсказывает активных существующих клиентов и очень плохо отток. Но AUC может оказаться достаточно высоким, ставим её в бой, а на деле - хорошо если не убытки для бизнеса...   <br>
    
<br><br>СПАСИБО)<br>
 УДАЧИ В ДАЛЬНЕЙШЕМ ОБУЧЕНИИ!!!

</b></font>

## 5. (бонус) Проверьте модели на адекватность

In [35]:
data['is_ultra'].value_counts()


0    2229
1     985
Name: is_ultra, dtype: int64

Для задач бинарной классификации могут быть использованы следующие методы:

Матрица ошибок / confusion matrix
Тесты бинарной классификации / binary classification tests
Коэффициент конверсии / conversion rates
ROC-кривая / ROC curve
Совокупный доход / cumulative gain
Lift-кривая / lift chart

Работает и сравнение модели со случайной.



In [36]:
df_test['is_ultra'].value_counts()


0    442
1    201
Name: is_ultra, dtype: int64

Доля большего класса тестовой выборки равна ~0.69; полученная нами модель имеет accuracy ~0.79. Таким образом, мы можем считать модель адекватной для использования в продакшене.



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
